In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [10]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("3-1.튜토리얼 따라하기")

LangSmith 추적을 시작합니다.
[프로젝트명]
3-1.튜토리얼 따라하기


출처 : https://wikidocs.net/251190

## 실습에 활용한 문서

- 소프트웨어정책연구소(SPRi) / 2025년 3월호

- 저자: 이해수(AI정책연구실 선임연구원), 유재흥(AI정책연구실 책임연구원)
- 링크: https://spri.kr/posts/view/23827?code=AI-Brief&s_year=&data_page=1
- 파일명: SPRI_AI_Brief_2023년12월호_F.pdf


In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

### 1. 문서로드

In [12]:
import os
path = os.getcwd()

In [13]:
loader = PyMuPDFLoader(f'{path}/data/SPRi AI Brief_3월호_산업동향_F.pdf')
docs = loader.load()
print(f'문서의 페이지 수 : {len(docs)}')

문서의 페이지 수 : 28


In [8]:
docs[:5]

[Document(metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-03-10T14:41:38+09:00', 'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf', 'file_path': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-03-10T14:41:38+09:00', 'trapped': '', 'modDate': "D:20250310144138+09'00'", 'creationDate': "D:20250310144138+09'00'", 'page': 0}, page_content='2025년\n3월호\n인공지능 산업의 최신 동향'),
 Document(metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-03-10T14:41:38+09:00', 'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf', 'file_path': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf', 'total_pages': 28,

In [9]:
docs[1].metadata

{'producer': 'Hancom PDF 1.3.0.505',
 'creator': 'Hancom PDF 1.3.0.505',
 'creationdate': '2025-03-10T14:41:38+09:00',
 'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf',
 'file_path': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf',
 'total_pages': 28,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2025-03-10T14:41:38+09:00',
 'trapped': '',
 'modDate': "D:20250310144138+09'00'",
 'creationDate': "D:20250310144138+09'00'",
 'page': 1}

### 2. 문서분할

In [14]:
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)
split_docs = text_spliter.split_documents(docs)
print(f'분할된 청크의 수 : {len(split_docs)}')

분할된 청크의 수 : 49


### 3. 임베딩 & DB생성
- 임베딩 : 과금 발생 구간 (문서가 크면 더 많은 과금)

In [15]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings) 

### 4. 검색기 생성
- 문서에 포함된 정보 검색 및 생성

In [16]:
retriever = vectorstore.as_retriever()

In [17]:
retriever.invoke("퍼플렉시티와 오픈AI 딥리서치 차이는?")

[Document(id='65a59f00-534f-4d4e-9542-df982a1cccf6', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-03-10T14:41:38+09:00', 'source': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf', 'file_path': '/Users/kimhyojin/Documents/langchain-kr/llm_rag_projict/data/SPRi AI Brief_3월호_산업동향_F.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-03-10T14:41:38+09:00', 'trapped': '', 'modDate': "D:20250310144138+09'00'", 'creationDate': "D:20250310144138+09'00'", 'page': 10}, page_content='정책･법제\n기업･산업\n기술･연구\n인력･교육\n9\n 오픈AI, 심층 조사를 수행하는 에이전트 ‘딥 리서치’ 공개\nn 오픈AI가 온라인 정보의 검색과 분석, 종합을 통해 전문가 수준의 종합 보고서를 작성하는 에이전트 \n‘딥 리서치’를 챗GPT 프로 사용자를 대상으로 출시 \nn 딥리서치는 HLE 벤치마크에서 26.6%의 정확도로 주요 모델 중 최고점을 기록했으나, 아직 \n초기 단계로 환각 증상이나 수식 오류 등의 문제점도 존재\nKEY Contents\n£ 오픈AI 딥 리서치, 온라인 정보 검색 및 분석을 통해 연구 분석가 수준의 보고서 작성\nn 오픈AI(OpenAI)가 2024년 2월 2일 추론을 통해 대

### 5. 프롬프트 생성

In [18]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)


### 6. 언어모델 생성

In [19]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [20]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
question = "퍼플렉시티와 오픈AI 딥리서치 차이는?"
response = chain.invoke(question)
print(response)

퍼플렉시티와 오픈AI의 딥 리서치의 주요 차이는 다음과 같습니다:

1. **가격 및 접근성**: 오픈AI의 딥 리서치는 유료 구독자에게만 제공되며, 초기에는 월 200달러의 구독료가 필요합니다. 반면, 퍼플렉시티의 딥 리서치는 무료로 제공되며, 무료 사용자도 하루 5개의 쿼리를 실행할 수 있습니다.

2. **정확도**: HLE 벤치마크에서 오픈AI의 딥 리서치는 26.6%의 정확도를 기록한 반면, 퍼플렉시티의 딥 리서치는 21.1%의 정확도를 기록했습니다.

3. **작업 완료 시간**: 퍼플렉시티의 딥 리서치는 대부분의 작업을 3분 이내에 완료하는 반면, 오픈AI의 딥 리서치는 작업 완료에 5~30분이 걸릴 수 있습니다.

이러한 차이점들이 퍼플렉시티와 오픈AI 딥 리서치의 주요 차이점입니다.
